In [1]:
!pip install mido pandas
!wget http://hog.ee.columbia.edu/craffel/lmd/lmd_full.tar.gz
!tar -xvzf lmd_full.tar.gz
!ls lmd_full  # Adjust the path if necessary
!wget https://colinraffel.com/projects/lmd/match_scores.json
!pip install pretty_midi
!pip install plotting
!wget https://www.ifs.tuwien.ac.at/mir/msd/partitions/msd-MAGD-genreAssignment.cls
import pretty_midi
import numpy as np
import joblib
import json
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os

Streaming output truncated to the last 5000 lines.
lmd_full/0/0f81ac2b48bf8db0cdad742baeac0926.mid
lmd_full/0/0385a9e793c3727b9b394926ccae1b00.mid
lmd_full/0/079e75b96c9f1631b422e592e9dd875f.mid
lmd_full/0/04489facaf997038a870bfa2ea1779cc.mid
lmd_full/0/0ecb96b6eaa6e3f0e2a3be0e166d419d.mid
lmd_full/0/0c88c69a6f9d74e08f0ebf69d5b9dabe.mid
lmd_full/0/08f8b64e242da1521f640cec850dcbd8.mid
lmd_full/0/0bba60265d9bca5f0a0c95a15a2e5fdb.mid
lmd_full/0/0aa00c786e5a06da5af9814a574d8625.mid
lmd_full/0/006e67220903376dce4bce993d373a73.mid
lmd_full/0/009a8dae33214c1233c3ca1d9e3d2f8d.mid
lmd_full/0/04bbf9725cdac1be555ed87517e5b62b.mid
lmd_full/0/06f27802a2668745a9916f8da8ab2080.mid
lmd_full/0/08b6fe7a3dc1cb81bc756808aef7f9ad.mid
lmd_full/0/0d81c05a115818f9ede97163612197ad.mid
lmd_full/0/087f6fee4c99aa1e781919fbab96bf4c.mid
lmd_full/0/02bdee19db2e884854cb8b36dc4d2e9e.mid
lmd_full/0/0e648210d826fa816bb2ace2afd15b6e.mid
lmd_full/0/03061af1932376c935b11319d9f7c4ca.mid
lmd_full/0/0103ae27fc14717507846ae1a6

In [2]:
def compute_statistics(midi_file):
    """
    Given a path to a MIDI file, compute a dictionary of statistics about it

    Parameters
    ----------
    midi_file : str
        Path to a MIDI file.

    Returns
    -------
    statistics : dict
        Dictionary reporting the values for different events in the file.
    """
    try:
        pm = pretty_midi.PrettyMIDI(midi_file)

        note_counts = [len(inst.notes) for inst in pm.instruments]
        pitch_ranges = [(min([note.pitch for note in inst.notes]),
                         max([note.pitch for note in inst.notes])) for inst in pm.instruments if inst.notes]
        avg_note_durations = [sum([note.end - note.start for note in inst.notes]) / len(inst.notes)
                              if inst.notes else 0 for inst in pm.instruments]
        avg_velocities = [sum([note.velocity for note in inst.notes]) / len(inst.notes)
                          if inst.notes else 0 for inst in pm.instruments]
        has_drums = any(inst.is_drum for inst in pm.instruments)

        melodic_intervals = []
        for inst in pm.instruments:
            if inst.notes:
                melodic_intervals += [abs(inst.notes[i + 1].pitch - inst.notes[i].pitch)
                                      for i in range(len(inst.notes) - 1)]
        avg_melodic_interval = sum(melodic_intervals) / len(melodic_intervals) if melodic_intervals else 0

        rhythmic_diversity = []
        for inst in pm.instruments:
            if inst.notes:
                rhythmic_diversity += [note.end - note.start for note in inst.notes]
        rhythmic_variability = (max(rhythmic_diversity) - min(rhythmic_diversity)
                                if rhythmic_diversity else 0)

        drum_patterns = []
        for inst in pm.instruments:
            if inst.is_drum and inst.notes:
                drum_patterns += [note.pitch for note in inst.notes]

        key_modulations = len(pm.key_signature_changes)


        return {
            'n_instruments': len(pm.instruments),     # Total number of instruments in the MIDI file
            'program_numbers': [i.program for i in pm.instruments if not i.is_drum],    # Program numbers for non-drum instruments, indicating their type (e.g., piano, guitar)
            'key_numbers': [k.key_number for k in pm.key_signature_changes], # Key numbers representing detected key signature changes
            # 'tempos': list(pm.get_tempo_changes()[1]), # List of tempos in the MIDI file (helps distinguish between slow and fast genres)
            # 'time_signature_changes': pm.time_signature_changes,  # List of time signature changes, if any (complex time signatures may indicate jazz/classical)
            'end_time': pm.get_end_time(), # Total duration of the piece in seconds
            # 'lyrics': [l.text for l in pm.lyrics], # Lyrics in the MIDI file, if available
            'note_counts': note_counts, # Number of notes played by each instrument
            # 'pitch_ranges': pitch_ranges, # Pitch range for each instrument as a tuple (lowest pitch, highest pitch)
            'avg_note_durations': avg_note_durations, # Average note durations for each instrument
            'avg_velocities': avg_velocities,  # Average velocities for each instrument
            'has_drums': has_drums,        # Whether the piece contains drums
            'drum_patterns': drum_patterns, # Extracted drum patterns, useful for analyzing rhythm sections
            'avg_melodic_interval': avg_melodic_interval, # Average melodic interval across all instruments
            'rhythmic_variability': rhythmic_variability, # Variability in note durations (useful for identifying rhythm complexity)
            'key_modulations': key_modulations, # Number of key modulations (changes in key signature)
            'tempo_estimate': pm.estimate_tempo() #numerical
        }
    except Exception as e:
        pass


In [3]:
def parse_genre_file(genre_file_path):
    genre_mapping = {}
    with open(genre_file_path, 'r') as f:
        for line in f:
            song_id, genre = line.strip().split('\t')
            genre_mapping[song_id] = genre
    return genre_mapping

# Load the genre mappings
genre_file_path = '/content/msd-MAGD-genreAssignment.cls'
genre_mapping = parse_genre_file(genre_file_path)


FileNotFoundError: [Errno 2] No such file or directory: '/content/msd-MAGD-genreAssignment.cls.txt'

In [ ]:
import json
import pandas as pd
import pretty_midi
with open('match_scores.json', 'r') as f:# open the file and make it a dictionary
    scores = json.load(f)

data_dict = []

for key in list(scores.keys())[:1000]: #loop through dictionary
    try:
      matches = scores[key]
      midi_md5 = list(matches.keys())[0] #Grabs the entire file name
      folder = midi_md5[0]  ##first char to find the folder
      file_path = f'lmd_full/{folder}/{midi_md5}.mid' #combine to get the file
      statistics = compute_statistics(file_path) #get stats of file

      song_id = key
      genre = genre_mapping.get(song_id, 'Unknown')
      statistics['genre'] = genre

      data_dict.append(statistics) # add it to a list
    except Exception as e:
      print(f'Error processing {key}: {e}')

# Create a DataFrame
df_dict = pd.DataFrame(data_dict) #create the df

df_dict = df_dict[df_dict['genre'] != 'Unknown']

# Save to CSV
df_dict.to_csv('dictionary_data.csv', index=False) #create a csv that can be used later
df_dict

In [ ]:
plt.hist(df_dict['n_instruments'])

In [ ]:
plt.hist(df_dict['tempo_estimate'])

In [ ]:
plt.hist(df_dict['program_numbers'])

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
import pandas as pd
import numpy as np

# Load the data
df_dict = pd.read_csv('dictionary_data.csv')

# Separate numeric and non-numeric data
numeric_cols = df_dict.select_dtypes(include=np.number).columns
non_numeric_cols = df_dict.select_dtypes(exclude=np.number).columns

# Handle numeric columns
numeric_imputer = SimpleImputer(strategy='most_frequent')
df_numeric = pd.DataFrame(
    numeric_imputer.fit_transform(df_dict[numeric_cols]),
    columns=numeric_cols
)

# Handle non-numeric columns
df_non_numeric = df_dict[non_numeric_cols].copy()

# Process specific non-numeric columns if needed (list like strings)
#for col in non_numeric_cols:
#    df_non_numeric[col] = df_non_numeric[col].apply(
#        lambda x: np.mean(eval(x)) if pd.notnull(x) and isinstance(x, str) and x.startswith("[") else np.nan
#   )

# Impute non-numeric columns
non_numeric_imputer = SimpleImputer(strategy='most_frequent')
df_non_numeric = pd.DataFrame(
    non_numeric_imputer.fit_transform(df_non_numeric),
    columns=non_numeric_cols
)

# Combine numeric and processed non-numeric columns
df_processed = pd.concat([df_numeric, df_non_numeric], axis=1)

# Scale numeric data
scaler = StandardScaler()
scaled_features = scaler.fit_transform(df_numeric)

# Feature expansion
poly = PolynomialFeatures(degree=2, include_bias=False)
expanded_features = poly.fit_transform(scaled_features)

# Convert back to DataFrame for expanded features
expanded_df = pd.DataFrame(expanded_features, columns=poly.get_feature_names_out(numeric_cols))

print(f"Preprocessed and expanded features: {expanded_df.shape}")
expanded_df['genre'] = df_dict['genre']
expanded_df

In [ ]:
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Scale the features (exclude 'genre' column from scaling)
# scaler = MinMaxScaler()
# df_norm = pd.DataFrame(scaler.fit_transform(expanded_df.drop(columns='genre')), columns=expanded_df.drop(columns='genre').columns)
df_norm = expanded_df.drop(columns='genre')

# Encode the 'genre' column (target) using LabelEncoder
label_encoder = LabelEncoder()
expanded_df['genre_encoded'] = label_encoder.fit_transform(expanded_df['genre'])

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df_norm, expanded_df['genre_encoded'], test_size=0.20, random_state=42)


# Train a linear SVM
svm_linear = SVC(kernel='linear')
svm_linear.fit(X_train, y_train)\

train_pred_linear = svm_linear.predict(X_train)
test_pred_linear = svm_linear.predict(X_test)

print('Linear SVM Classification Report:')
print(classification_report(y_test, test_pred_linear))

# Train an RBF SVM
svm_rbf = SVC(kernel='rbf')
svm_rbf.fit(X_train, y_train)
train_pred_rbf = svm_linear.predict(X_train)
test_pred_rbf = svm_rbf.predict(X_test)

print('RBF SVM Classification Report:')
print(classification_report(y_test, test_pred_rbf))

In [ ]:
# Evaluate the model
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import accuracy_score

train_accuracy = mse(y_train, train_pred_linear)
test_accuracy = mse(y_test, test_pred_linear)
train_accuracy_acc = accuracy_score(y_train, train_pred_linear)
test_accuracy_acc = accuracy_score(y_test, test_pred_linear)

print(f"Training Accuracy Linear MSE: {train_accuracy:.2f}")
print(f"Testing Accuracy Linear MSE: {test_accuracy:.2f}")
print(f"Training Accuracy Linear ACC: {train_accuracy_acc:.2f}")
print(f"Testing Accuracy Linear ACC: {test_accuracy_acc:.2f}")

train_accuracy_rbf = mse(y_train, train_pred_rbf)
test_accuracy_rbf = mse(y_test, test_pred_rbf)
train_accuracy_rbf_acc = accuracy_score(y_train, train_pred_rbf)
test_accuracy_rbf_acc = accuracy_score(y_test, test_pred_rbf)

print(f"Training Accuracy RBF MSE: {train_accuracy_rbf:.2f}")
print(f"Testing Accuracy RBF MSE: {test_accuracy_rbf:.2f}")
print(f"Training Accuracy RBF ACC: {train_accuracy_rbf_acc:.2f}")
print(f"Testing Accuracy RBF ACC: {test_accuracy_rbf_acc:.2f}")


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set up the plotting style
sns.set(style="whitegrid")

# Prepare a list of numeric features (exclude genre and non-numeric columns)
numeric_features = df_dict.select_dtypes(include=['float64', 'int64']).columns.tolist()
numeric_features = [col for col in numeric_features if col != 'genre_encoded']

# Create box plots for each numeric feature against genre
for feature in numeric_features:
    plt.figure(figsize=(10, 6))
    sns.boxplot(x='genre', y=feature, data=df_dict)
    plt.xticks(rotation=45, ha='right')  # Rotate genre labels for readability
    plt.title(f'{feature} vs Genre')
    plt.ylabel(feature)
    plt.xlabel('Genre')
    plt.tight_layout()
    plt.show()
